In [1]:
!pip -q install google-genai

!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [2]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
# --- Configuração Inicial e Importações ---
# Assumindo que você já instalou google-genai: %pip -q install google-genai
# E configurou sua API Key nos Secrets do Colab

import os
import json # Precisamos de json para trabalhar com a saída simulada da "ferramenta" Spotify
import textwrap # Para formatar a saída Markdown
from google.colab import userdata
from google import genai # Importa a biblioteca genai diretamente
from IPython.display import display, Markdown # Para exibir texto formatado no Colab

# --- Configura a API Key do Google Gemini ---
# Certifique-se de que 'GOOGLE_API_KEY' está nos Secrets do seu Colab
# Se esta célula falhar, verifique seus Secrets do Colab.
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    print("API Key configurada com sucesso.")
except Exception as e:
    print(f"Erro ao configurar API Key: {e}")
    print("Por favor, verifique se sua GOOGLE_API_KEY está nos Secrets do Colab.")

# --- Configura o cliente da SDK do Gemini ---
# Esta instância será usada para todas as chamadas ao modelo
try:
    client = genai.Client()
    print("Cliente GenAI configurado com sucesso.")
except Exception as e:
    print(f"Erro ao configurar cliente GenAI: {e}")
    # Se a API Key falhou acima, esta configuração também falhará

# --- Funções Auxiliares ---

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
    # Substitui marcadores de lista comuns por formato Markdown
    text = text.replace('•', '  *')
    # Adiciona identação para destacar a saída no display (opcional, mas útil)
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Função auxiliar para chamar o modelo GenAI
# Substitui a lógica de 'call_agent' do ADK
def get_model_response(prompt_text: str, model_name: str = "gemini-1.5-flash-latest") -> str:
    """
    Chama o modelo Gemini para gerar conteúdo com base em um prompt.

    Args:
        prompt_text: O texto do prompt a ser enviado ao modelo.
        model_name: O nome do modelo a ser usado (padrão: gemini-1.5-flash-latest).

    Returns:
        A string de resposta do modelo, ou uma mensagem de erro/aviso.
    """
    if not client:
        return "Erro: Cliente GenAI não configurado."

    try:
        # Chamada direta ao modelo usando a biblioteca google-genai
        response = client.models.generate_content(
            model=model_name,
            contents=prompt_text
        )

        # Verifica se a resposta contém texto válido
        if response.candidates and response.candidates[0].content.parts:
             # Concatena todas as partes de texto da resposta
             return "".join(part.text for part in response.candidates[0].content.parts if part.text is not None)
        else:
             # Lida com casos onde a resposta está vazia ou contém outros dados (ex: bloqueios de segurança)
             warning_message = f"Aviso: Resposta do modelo vazia ou inesperada."
             if response.prompt_feedback and response.prompt_feedback.block_reason:
                 warning_message += f" Motivo do bloqueio do prompt: {response.prompt_feedback.block_reason}."
             if response.candidates and response.candidates[0].finish_reason:
                  warning_message += f" Motivo de término do candidato: {response.candidates[0].finish_reason}."
             if response.candidates and response.candidates[0].safety_ratings:
                 warning_message += f" Avaliações de segurança: {response.candidates[0].safety_ratings}."

             print(warning_message)
             return f"Não foi possível gerar uma resposta: {warning_message}" # Indica falha

    except Exception as e:
        # Captura erros na comunicação com a API
        error_message = f"Erro ao chamar o modelo {model_name}: {e}"
        print(error_message)
        return f"Erro na comunicação com a IA: {error_message}" # Indica falha

# --- Funções que Simulam o Comportamento dos Agentes ---
# Cada função chama o modelo com um prompt específico para realizar uma tarefa.

# Função 1: Simula o Agente Analisador Emocional
def analisar_emocao_genai(texto_usuario: str) -> str:
    """
    Analisa o estado emocional do usuário a partir de texto usando o modelo Gemini
    e formata a saída para ser usada pela próxima etapa.
    """
    prompt = f"""
    Você é um analisador de estado emocional e humor, com o objetivo de ajudar a selecionar músicas.
    Analise o texto fornecido pelo usuário que descreve como ele se sente e identifique o estado emocional ou humor predominante.
    Com base no humor, sugira características musicais (gêneros, ritmo, energia, atmosfera, instrumentalização, tema lírico) que combinem com esse estado.

    Formate sua resposta DESTA forma para facilitar o processamento na próxima etapa:
    Humor Detectado: [Liste aqui os humores e nuances identificados. Ex: 'Melancólico, um pouco entediado']
    Características Musicais Sugeridas: [Liste aqui as características musicais. Ex: 'Músicas introspectivas, talvez Folk acústico ou Jazz suave, ritmo lento a moderado']

    Se o texto for ambíguo ou não descrever um humor claro, formate a resposta como:
    Humor Detectado: Não identificado
    Características Musicais Sugeridas: Não aplicável

    Texto do usuário para analisar:
    "{texto_usuario}"
    """
    print("\n--- 🧠 Iniciando Análise Emocional (GenAI Direto)... ---")
    resultado_analise = get_model_response(prompt)
    print("--- ✅ Análise Concluída. ---")
    return resultado_analise

# Função 2: Simula a Chamada da Ferramenta Spotify + Refinamento (Combinado e Simulado)
def simular_chamada_spotify_genai(analise_emocional_str: str) -> str:
    """
    Simula a obtenção de sugestões musicais com base na análise emocional
    e simula uma busca/recomendação no Spotify.
    Em uma versão real, isso chamaria a API do Spotify.
    """
    # Nesta simulação, o próprio modelo vai "simular" o comportamento da ferramenta.
    # Em uma versão real, você PARSARIA analise_emocional_str aqui no Python
    # e usaria os dados para chamar a API do Spotify real.

    prompt = f"""
    Você está simulando ser uma ferramenta de busca e recomendação de músicas, agindo como se estivesse interagindo com uma API como a do Spotify.
    Recebeu a seguinte análise emocional e características musicais:

    {analise_emocional_str}

    Com base no "Humor Detectado" e nas "Características Musicais Sugeridas" da análise acima, simule uma busca por 10 músicas que se encaixem.
    Simule uma resposta como se viesse de uma API, retornando uma lista de títulos de músicas e seus artistas.

    Formate sua resposta EXATAMENTE como um objeto JSON com a seguinte estrutura, e não inclua nenhum texto antes ou depois do JSON:
    {{
      "status": "sucesso_simulado",
      "mensagem": "Busca simulada concluída com base na análise emocional.",
      "musicas_sugeridas": [
        "Título da Música 1 - Artista 1",
        "Título da Música 2 - Artista 2",
        "Título da Música 3 - Artista 3",
        "Título da Música 4 - Artista 4",
        "Título da Música 5 - Artista 5",
        "Título da Música 6 - Artista 6",
        "Título da Música 7 - Artista 7",
        "Título da Música 8 - Artista 8",
        "Título da Música 9 - Artista 9",
        "Título da Música 10 - Artista 10"
      ]
    }}

    Se a análise emocional indicar que o humor não foi identificado ou que o texto não é relevante para música, retorne um JSON com uma lista vazia ou uma mensagem de erro na lista:
    {{
      "status": "falha_simulada",
      "mensagem": "Humor não identificado ou irrelevante para sugestões musicais.",
      "musicas_sugeridas": []
    }}

    Não inclua nenhum texto explicativo antes ou depois do bloco JSON. Apenas o JSON.
    """
    # Usamos um modelo que tende a ser bom em seguir instruções de formato (JSON)
    print("\n--- 🎵 Iniciando Simulação de Busca por Sugestões (GenAI Direto)... ---")
    simulated_json_str = get_model_response(prompt, model_name="gemini-1.5-flash-latest") # Ou "gemini-1.0-pro" se precisar de mais confiabilidade em JSON

    # Tenta limpar e parsear a resposta simulada como JSON
    try:
        # Remove potenciais blocos de código Markdown (```json...```)
        simulated_json_str = simulated_json_str.strip()
        if simulated_json_str.startswith("```json"):
            simulated_json_str = simulated_json_str[len("```json"):].strip()
        if simulated_json_str.endswith("```"):
            simulated_json_str = simulated_json_str[:-len("```")].strip()

        simulated_result = json.loads(simulated_json_str)

        # Valida se a estrutura mínima esperada está presente
        if isinstance(simulated_result, dict) and "musicas_sugeridas" in simulated_result and isinstance(simulated_result["musicas_sugeridas"], list):
             print("--- ✅ Simulação de busca concluída e JSON válido. ---")
             # Retorna a string JSON válida para a próxima etapa
             return json.dumps({"musicas_sugeridas": simulated_result["musicas_sugeridas"]})
        else:
             # JSON válido, mas estrutura inesperada
             print(f"--- ❌ Simulação de busca concluída, mas estrutura JSON inesperada. ---")
             print(f"Resposta JSON recebida: {simulated_json_str}")
             return json.dumps({"musicas_sugeridas": [f"Erro ao processar JSON simulado: Estrutura inesperada. Resposta: {simulated_json_str[:200]}..."]}) # Indica falha
    except json.JSONDecodeError as e:
        # Erro ao decodificar o JSON
        print(f"--- ❌ Erro ao decodificar JSON simulado: {e} ---")
        print(f"Resposta simulada bruta recebida: {simulated_json_str}")
        return json.dumps({"musicas_sugeridas": [f"Erro ao decodificar resposta simulada: {e}. Resposta bruta: {simulated_json_str[:200]}..."]}) # Indica falha
    except Exception as e:
         # Captura outros erros inesperados
         print(f"--- ❌ Erro inesperado na simulação: {e} ---")
         return json.dumps({"musicas_sugeridas": [f"Erro inesperado na simulação: {e}"]}) # Indica falha


# Função 3: Simula o Agente Formatador de Playlist
def formatar_playlist_genai(simulated_suggestions_json_str: str) -> str:
    """
    Formata a lista de músicas sugeridas (obtida da simulação) em um texto amigável
    para o usuário, simulando a criação final da playlist.
    """
    try:
        # Carrega os dados da simulação (que vêm como string JSON)
        simulated_data = json.loads(simulated_suggestions_json_str)
        sugestoes = simulated_data.get("musicas_sugeridas", [])

        # Verifica se há sugestões válidas para formatar
        if not sugestoes or (isinstance(sugestoes[0], str) and sugestoes[0].startswith("Erro")):
             # Se a lista estiver vazia ou contiver uma mensagem de erro da simulação anterior
             print("\n--- 📝 Sem sugestões válidas para formatar. ---")
             return "\nNão foi possível gerar sugestões de músicas com base no seu humor. Tente descrever como se sente de outra forma."

        # Formata a lista de sugestões como texto para o prompt
        sugestoes_list_str = "\n".join([f"- {item}" for item in sugestoes])

        prompt = f"""
        Você é um criador de posts e curador musical, especialista em apresentar sugestões de músicas de forma atraente para um usuário, como se fosse uma playlist.
        Recebeu a seguinte lista de músicas sugeridas:

        {sugestoes_list_str}

        Sua tarefa é transformar esta lista em um texto curto e envolvente para o usuário.
        Comece com uma saudação amigável e uma frase que conecte com o humor (baseado nas sugestões listadas, não na análise emocional original), apresente as músicas como uma lista fácil de ler com marcadores (bullet points), e termine com uma breve conclusão, um convite para ouvir.
        Não mencione que a lista é simulada, que veio de uma ferramenta ou de uma IA. Apresente como sugestões diretas e personalizadas para ele.
        Use uma linguagem amigável e casual.

        Exemplo de introdução: "Com base no que você me contou, preparei algumas músicas que combinam com essa vibe:"
        Exemplo de conclusão: "Espero que goste desta seleção!"
        """
        print("\n--- 📝 Iniciando Formatação da Lista de Sugestões (GenAI Direto)... ---")
        playlist_texto = get_model_response(prompt)
        print("--- ✅ Formatação concluída. ---")
        return playlist_texto

    except json.JSONDecodeError as e:
         print(f"--- ❌ Erro ao decodificar JSON para formatação: {e} ---")
         return f"\nErro interno ao processar sugestões para formatação: {e}"
    except Exception as e:
         print(f"--- ❌ Erro inesperado na formatação: {e} ---")
         return f"\nErro inesperado ao formatar sugestões: {e}"


# --- Bloco Principal de Execução (Orquestração Manual) ---

# Assumindo que as células de configuração de API Key e cliente GenAI já foram executadas.
# Execute este bloco após a configuração inicial e as definições das funções acima.
if __name__ == '__main__':
    print("👋 Olá! Bem-vindo ao Analisador de Humor Musical (GenAI Direto).")
    print("Este sistema sugere músicas com base no seu estado emocional.")

    # --- Passo 1: Obter e Analisar o Humor do Usuário ---
    texto_do_usuario = input("❓ Como você está se sentindo agora? Descreva seu estado emocional: ")

    if not texto_do_usuario:
        print("❌ Você não digitou nada. Por favor, me diga como se sente na próxima vez.")
    else:
        print(f"\nOk, vou analisar seu sentimento: '{texto_do_usuario}'")

        # Chama a função que simula o Agente Analisador Emocional
        analise_emocional_str = analisar_emocao_genai(texto_do_usuario)

        # Verifica se a análise emocional retornou uma mensagem de erro ou 'Não identificado'
        if "Erro:" in analise_emocional_str or "Não foi possível gerar uma resposta" in analise_emocional_str or "Humor Detectado: Não identificado" in analise_emocional_str:
             print("\n❌ Não foi possível analisar seu humor ou ele não foi identificado.")
             display(to_markdown(analise_emocional_str)) # Exibe o motivo do erro/não identificação
        else:
            print("\n--- Resultado da Análise Emocional ---")
            display(to_markdown(analise_emocional_str))
            print("---------------------------------------")

            # --- Passo 2: Simular a Busca por Sugestões Musicais ---
            # Passa o resultado da análise emocional para a função que simula a busca/recomendação
            simulated_suggestions_json_str = simular_chamada_spotify_genai(analise_emocional_str)

            # --- Passo 3: Formatar a Lista de Sugestões e Exibir o Resultado Final ---
            # Passa o resultado (string JSON) da simulação para a função de formatação
            playlist_texto_final = formatar_playlist_genai(simulated_suggestions_json_str)

            # Exibe o resultado final formatado para o usuário
            print("\n--- Sua Playlist Sugerida ---")
            display(to_markdown(playlist_texto_final))
            print("-----------------------------")


    print("\n--- Fim da execução do sistema (GenAI Direto). ---")

API Key configurada com sucesso.
Cliente GenAI configurado com sucesso.
👋 Olá! Bem-vindo ao Analisador de Humor Musical (GenAI Direto).
Este sistema sugere músicas com base no seu estado emocional.
❓ Como você está se sentindo agora? Descreva seu estado emocional: Está chuvoso e me sinto triste.

Ok, vou analisar seu sentimento: 'Está chuvoso e me sinto triste.'

--- 🧠 Iniciando Análise Emocional (GenAI Direto)... ---
--- ✅ Análise Concluída. ---

--- Resultado da Análise Emocional ---


> Humor Detectado: Triste, melancólico
> 
> Características Musicais Sugeridas: Músicas com atmosfera introspectiva e melancólica.  Gêneros como Indie Folk,  Música Clássica (especialmente peças para piano ou violino), ou até mesmo alguns tipos de música eletrônica mais ambiente e downtempo poderiam ser apropriados. Ritmo lento a moderado. Instrumentação predominantemente acústica (violão, piano, cordas) ou eletrônica minimalista. Temas líricos relacionados à solidão, reflexão, saudade ou a própria chuva seriam coerentes.


---------------------------------------

--- 🎵 Iniciando Simulação de Busca por Sugestões (GenAI Direto)... ---
--- ✅ Simulação de busca concluída e JSON válido. ---

--- 📝 Iniciando Formatação da Lista de Sugestões (GenAI Direto)... ---
--- ✅ Formatação concluída. ---

--- Sua Playlist Sugerida ---


> E aí, tudo bem?  Preparei uma playlist que vai te levar pra um lugar bem zen, mas com uns momentos de sofrência deliciosa, sabe?  Aquele mix perfeito pra relaxar e refletir, sem cair no tédio!  Aperta o play e embarca nessa viagem sonora:
> 
> * Hallelujah - Jeff Buckley (a versão definitiva, você sabe)
> * Skinny Love - Bon Iver (pra sentir aquela emoção gostosa na alma)
> * Flowers in Your Hair - The Lumineers (um toque de alegria pra equilibrar a balança)
> * Nuvole Bianche - Ludovico Einaudi (prepare as lágrimas, de tão lindo)
> * Watermark - Enya (uma imersão total em paz e tranquilidade)
> * A Case of You - Joni Mitchell (clássico atemporal pra suspirar)
> * Shelter Song - Temples (vibes psicodélicas pra viajar na maionese)
> * The Scientist - Coldplay (pra acalmar o coração e refletir)
> * Comptine D'un Autre Été: L'après-midi - Yann Tiersen (uma trilha sonora de filme que vai te transportar)
> * Riverside - Agnes Obel (voz angelical pra encerrar com chave de ouro)
> 
> 
> Espero que goste desta seleção!  Me conta depois o que achou!


-----------------------------

--- Fim da execução do sistema (GenAI Direto). ---
